In [2]:
"""
Interactive demonstration for Bayesian‑perception .

Controls
--------
s range : –5 … 5  (true stimulus value)
σ range  : 0.1 … 3.0 (measurement‑noise SD)
Samples  : 5 … 200 measurements per draw

The x‑axis is fixed to [–17, 17].  The y‑axis is fixed so the likelihood
peak always tops out around y = 0.15.
"""

import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider, IntSlider, Checkbox, VBox, HBox
from IPython.display import display

# Make static images sharp inside notebooks
%matplotlib inline
plt.rcParams["figure.dpi"] = 120

# ---------------------------------------------------------------------------
# Global axis limits (stay *constant* no matter what the sliders do)
S0_RANGE = (-5, 5)          # from sliders
SIGMA_RANGE = (0.1, 3.0)    # from sliders
XLIM = (S0_RANGE[0] - 4 * SIGMA_RANGE[1],
        S0_RANGE[1] + 4 * SIGMA_RANGE[1])   # (-17, 17)
YLIM = (-0.05, 0.2)         # room for baseline + PDF peak (0.15)


def measurement_demo(s0: float = 0.0, sigma: float = 1.0,
                     n_samples: int = 50, show_pdf: bool = True):
    
    """Draw `n_samples` measurements from N(s₀, σ²) and plot them.
    """
    rng = np.random.default_rng()
    samples = rng.normal(loc=s0, scale=sigma, size=n_samples)

    fig, ax = plt.subplots(figsize=(6, 3))

    # True stimulus line
    ax.axvline(s0, color="black", linestyle="--", label=r"True stimulus $s_0$")

    # Measurement samples
    ax.scatter(samples, np.zeros_like(samples), alpha=0.6,
               label="Measurements $x_i$", zorder=3)

    # Likelihood curve (scaled)
    if show_pdf:
        xs = np.linspace(XLIM[0], XLIM[1], 400)
        pdf = (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * ((xs - s0) / sigma) ** 2)
        pdf_scaled = pdf / pdf.max() * 0.15 # Scaling the pdf for better visual
        ax.plot(xs, pdf_scaled, color="red", linewidth=2,
                label=r"Likelihood $p(x\,|\,s_0)$")

    # Fixed axes
    ax.set_xlim(*XLIM)
    ax.set_ylim(*YLIM)

    # Aesthetics
    ax.set_xlabel(r"Measurement value $x$")
    ax.set_yticks([])
    ax.set_title(f"{n_samples} measurements, σ = {sigma:.2f}")
    ax.legend(loc="upper right", framealpha=0.9)
    ax.grid(True, axis="x", linestyle=":", alpha=0.5)
    plt.show()


# ---------------------------------------------------------------------------
# controls
controls = {
    "s0": FloatSlider(value=0.0, min=S0_RANGE[0], max=S0_RANGE[1], step=0.1,
                       description="Stimulus $s_0$:", continuous_update=True,
                       readout_format=".1f"),
    "sigma": FloatSlider(value=1.0, min=SIGMA_RANGE[0], max=SIGMA_RANGE[1], step=0.1,
                          description="Noise σ:", continuous_update=True),
    "n_samples": IntSlider(value=50, min=5, max=200, step=1,
                           description="# samples:", continuous_update=True),
    "show_pdf": Checkbox(value=True, description="Show likelihood curve"),
}

interactive_plot = interactive(measurement_demo, **controls)

# Two‑column layout: sliders left, plot right
ui_left = VBox([controls["s0"], controls["sigma"],
                controls["n_samples"], controls["show_pdf"]])
output_area = interactive_plot.children[-1]

display(HBox([ui_left, output_area]))
interactive_plot.update()
